# Chapter 6: Metaclasses and AttributesThis chapter covers advanced Python features for customizing attribute access and class creation.

## Item 44: Use Plain Attributes Instead of Setter and Getter Methods### Key ConceptAlways start with simple public attributes. Only add @property when you need special behavior.

In [ ]:
# ❌ NOT PYTHONIC - Explicit getters/settersclass OldResistor:    def __init__(self, ohms):        self._ohms = ohms        def get_ohms(self):        return self._ohms        def set_ohms(self, ohms):        self._ohms = ohmsr0 = OldResistor(50e3)print('Before:', r0.get_ohms())r0.set_ohms(10e3)print('After:', r0.get_ohms())

In [ ]:
# ✅ PYTHONIC - Simple public attributesclass Resistor:    def __init__(self, ohms):        self.ohms = ohms        self.voltage = 0        self.current = 0r1 = Resistor(50e3)r1.ohms = 10e3r1.ohms += 5e3print(f'Resistance: {r1.ohms} ohms')

### Using @property for Special Behavior

In [ ]:
class VoltageResistance(Resistor):    def __init__(self, ohms):        super().__init__(ohms)        self._voltage = 0        @property    def voltage(self):        return self._voltage        @voltage.setter    def voltage(self, voltage):        self._voltage = voltage        self.current = self._voltage / self.ohmsr2 = VoltageResistance(1e3)print(f'Before: {r2.current:.2f} amps')r2.voltage = 10print(f'After: {r2.current:.2f} amps')

### Validation with @property

In [ ]:
class BoundedResistance(Resistor):    def __init__(self, ohms):        super().__init__(ohms)        @property    def ohms(self):        return self._ohms        @ohms.setter    def ohms(self, ohms):        if ohms <= 0:            raise ValueError(f'ohms must be > 0; got {ohms}')        self._ohms = ohmsr3 = BoundedResistance(1e3)print(f'Valid: {r3.ohms}')try:    r3.ohms = 0except ValueError as e:    print(f'Error: {e}')

---## Item 45: Consider @property Instead of Refactoring Attributes### Migrating to Calculated Properties

In [ ]:
from datetime import datetime, timedeltaclass Bucket:    def __init__(self, period):        self.period_delta = timedelta(seconds=period)        self.reset_time = datetime.now()        self.quota = 0        def __repr__(self):        return f'Bucket(quota={self.quota})'def fill(bucket, amount):    now = datetime.now()    if (now - bucket.reset_time) > bucket.period_delta:        bucket.quota = 0        bucket.reset_time = now    bucket.quota += amountdef deduct(bucket, amount):    now = datetime.now()    if (now - bucket.reset_time) > bucket.period_delta:        return False    if bucket.quota - amount < 0:        return False    bucket.quota -= amount    return Truebucket = Bucket(60)fill(bucket, 100)print(bucket)if deduct(bucket, 99):    print('Had 99 quota')print(bucket)

### Improved Version with @property

In [ ]:
class NewBucket:    def __init__(self, period):        self.period_delta = timedelta(seconds=period)        self.reset_time = datetime.now()        self.max_quota = 0        self.quota_consumed = 0        def __repr__(self):        return (f'NewBucket(max_quota={self.max_quota}, '                f'quota_consumed={self.quota_consumed})')        @property    def quota(self):        return self.max_quota - self.quota_consumed        @quota.setter    def quota(self, amount):        delta = self.max_quota - amount        if amount == 0:            self.quota_consumed = 0            self.max_quota = 0        elif delta < 0:            assert self.quota_consumed == 0            self.max_quota = amount        else:            assert self.max_quota >= self.quota_consumed            self.quota_consumed += deltabucket = NewBucket(60)print('Initial', bucket)fill(bucket, 100)print('Filled', bucket)

---## Item 46: Use Descriptors for Reusable @property Methods### The Problem: @property Can't Be Reused

In [ ]:
# Problem: Repetitive @property codeclass Homework:    def __init__(self):        self._grade = 0        @property    def grade(self):        return self._grade        @grade.setter    def grade(self, value):        if not (0 <= value <= 100):            raise ValueError('Grade must be between 0 and 100')        self._grade = valuegalileo = Homework()galileo.grade = 95print(f'Grade: {galileo.grade}')

### Solution: Descriptors

In [ ]:
from weakref import WeakKeyDictionaryclass Grade:    def __init__(self):        self._values = WeakKeyDictionary()        def __get__(self, instance, instance_type):        if instance is None:            return self        return self._values.get(instance, 0)        def __set__(self, instance, value):        if not (0 <= value <= 100):            raise ValueError('Grade must be between 0 and 100')        self._values[instance] = valueclass Exam:    math_grade = Grade()    writing_grade = Grade()    science_grade = Grade()first_exam = Exam()first_exam.writing_grade = 82second_exam = Exam()second_exam.writing_grade = 75print(f'First: {first_exam.writing_grade}')print(f'Second: {second_exam.writing_grade}')

---## Item 47: Use __getattr__, __getattribute__, and __setattr__ for Lazy Attributes### __getattr__ for Lazy Loading

In [ ]:
class LazyRecord:    def __init__(self):        self.exists = 5        def __getattr__(self, name):        value = f'Value for {name}'        setattr(self, name, value)        return valuedata = LazyRecord()print('Before:', data.__dict__)print('foo:', data.foo)print('After:', data.__dict__)

### __getattribute__ for All Access

In [ ]:
class ValidatingRecord:    def __init__(self):        self.exists = 5        def __getattribute__(self, name):        print(f'* Called __getattribute__({name!r})')        try:            value = super().__getattribute__(name)            print(f'* Found {name!r}, returning {value!r}')            return value        except AttributeError:            value = f'Value for {name}'            print(f'* Setting {name!r} to {value!r}')            setattr(self, name, value)            return valuedata = ValidatingRecord()print('exists:', data.exists)print('First foo:', data.foo)print('Second foo:', data.foo)

### __setattr__ for Write Interception

In [ ]:
class LoggingSavingRecord:    def __setattr__(self, name, value):        print(f'* Called __setattr__({name!r}, {value!r})')        super().__setattr__(name, value)data = LoggingSavingRecord()print('Before:', data.__dict__)data.foo = 5print('After:', data.__dict__)data.foo = 7print('Finally:', data.__dict__)

---## Item 48: Validate Subclasses with __init_subclass__### Metaclass Approach (Old Way)

In [ ]:
class Meta(type):    def __new__(meta, name, bases, class_dict):        print(f'* Running {meta}.__new__ for {name}')        print('Bases:', bases)        return type.__new__(meta, name, bases, class_dict)class MyClass(metaclass=Meta):    stuff = 123    def foo(self):        passclass MySubclass(MyClass):    other = 567    def bar(self):        pass

### __init_subclass__ Approach (Better)

In [ ]:
class BetterPolygon:    sides = None        def __init_subclass__(cls):        super().__init_subclass__()        if cls.sides < 3:            raise ValueError('Polygons need 3+ sides')        @classmethod    def interior_angles(cls):        return (cls.sides - 2) * 180class Hexagon(BetterPolygon):    sides = 6print(f'Hexagon angles: {Hexagon.interior_angles()}')try:    class Point(BetterPolygon):        sides = 1except ValueError as e:    print(f'Error: {e}')

### Multiple Inheritance with __init_subclass__

In [ ]:
class Filled:    color = None        def __init_subclass__(cls):        super().__init_subclass__()        if cls.color not in ('red', 'green', 'blue'):            raise ValueError('Fills need a valid color')class RedTriangle(Filled, BetterPolygon):    color = 'red'    sides = 3ruddy = RedTriangle()print(f'Created: {ruddy.__class__.__name__}')

---## Item 49: Register Class Existence with __init_subclass__### Automatic Class Registration

In [ ]:
import jsonclass Serializable:    def __init__(self, *args):        self.args = args        def serialize(self):        return json.dumps({'args': self.args})class Point2D(Serializable):    def __init__(self, x, y):        super().__init__(x, y)        self.x = x        self.y = y        def __repr__(self):        return f'Point2D({self.x}, {self.y})'point = Point2D(5, 3)print('Object:', point)print('Serialized:', point.serialize())

### With __init_subclass__ Registration

In [ ]:
registry = {}class BetterSerializable:    def __init__(self, *args):        self.args = args        def __init_subclass__(cls):        super().__init_subclass__()        registry[cls.__name__] = cls        def serialize(self):        return json.dumps({            'class': self.__class__.__name__,            'args': self.args        })        def __repr__(self):        name = self.__class__.__name__        args_str = ', '.join(str(x) for x in self.args)        return f'{name}({args_str})'def deserialize(data):    params = json.loads(data)    name = params['class']    target_class = registry[name]    return target_class(*params['args'])class Vector1D(BetterSerializable):    def __init__(self, magnitude):        super().__init__(magnitude)        self.magnitude = magnitudebefore = Vector1D(6)data = before.serialize()after = deserialize(data)print(f'Before: {before}')print(f'After: {after}')

---## Item 50: Annotate Class Attributes with __set_name__### Descriptor with __set_name__

In [ ]:
class Field:    def __init__(self):        self.name = None        self.internal_name = None        def __set_name__(self, owner, name):        self.name = name        self.internal_name = '_' + name        def __get__(self, instance, instance_type):        if instance is None:            return self        return getattr(instance, self.internal_name, '')        def __set__(self, instance, value):        setattr(instance, self.internal_name, value)class Customer:    first_name = Field()    last_name = Field()    prefix = Field()    suffix = Field()cust = Customer()print('Before:', cust.__dict__)cust.first_name = 'Euclid'print('After:', cust.__dict__)

---## Item 51: Prefer Class Decorators Over Metaclasses### Class Decorator for Tracing

In [ ]:
from functools import wrapsdef trace_func(func):    if hasattr(func, 'tracing'):        return func        @wraps(func)    def wrapper(*args, **kwargs):        result = None        try:            result = func(*args, **kwargs)            return result        except Exception as e:            result = e            raise        finally:            print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')        wrapper.tracing = True    return wrapperdef trace(klass):    for key in dir(klass):        value = getattr(klass, key)        if callable(value):            wrapped = trace_func(value)            setattr(klass, key, wrapped)    return klass@traceclass TraceDict(dict):    passtrace_dict = TraceDict([('hi', 1)])trace_dict['there'] = 2trace_dict['hi']

---## Chapter Summary### Key Concepts| Item | Concept | Use Case ||------|---------|----------|| 44 | Plain attributes + @property | Start simple, add behavior later || 45 | @property for refactoring | Migrate without breaking API || 46 | Descriptors | Reusable @property logic || 47 | __getattr__, __setattr__ | Lazy loading, proxies || 48 | __init_subclass__ | Subclass validation || 49 | Class registration | Automatic type registry || 50 | __set_name__ | Descriptor introspection || 51 | Class decorators | Composable class extensions |